In [40]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()


Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Fetched 252 kB in 1s (178 kB/s)
Reading package lis

In [41]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-07-12 17:50:38--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.3’

postgresql-42.2.16. 100%[===================>] 979.38K  1.60MB/s    in 0.6s    

2022-07-12 17:50:40 (1.60 MB/s) - ‘postgresql-42.2.16.jar.3’ saved [1002883/1002883]



In [42]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Final Project").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [43]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://plasticpollution.s3.amazonaws.com/Plastic_waste_mismanagement_clean.csv"
spark.sparkContext.addFile(url)
Plastic_waste_mismanagement_df = spark.read.csv(SparkFiles.get("Plastic_waste_mismanagement_clean.csv"), sep=",", header=True, inferSchema=True)

In [44]:
# Show DataFrame
Plastic_waste_mismanagement_df.show()    

+--------------------+----+----+-------------------------+
|             Country|Code|Year|PlasticWaste(metric tons)|
+--------------------+----+----+-------------------------+
|              Africa|null|2019|                 13690104|
|             Albania| ALB|2019|                    69833|
|             Algeria| DZA|2019|                   764578|
|              Angola| AGO|2019|                   236946|
| Antigua and Barbuda| ATG|2019|                      627|
|           Argentina| ARG|2019|                   465808|
|                Asia|null|2019|                 39881167|
|           Australia| AUS|2019|                     5266|
|             Bahamas| BHS|2019|                     2212|
|             Bahrain| BHR|2019|                     1043|
|          Bangladesh| BGD|2019|                  1021990|
|            Barbados| BRB|2019|                      872|
|             Belgium| BEL|2019|                     2284|
|              Belize| BLZ|2019|                     602

In [45]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://plasticpollution.s3.amazonaws.com/pbt_pivoted_clean.csv"
spark.sparkContext.addFile(url)
pbt_df = spark.read.csv(SparkFiles.get("pbt_pivoted_clean.csv"), sep=",", header=True, inferSchema=True)

In [46]:
# Show DataFrame
pbt_df.show()    

+--------+--------+--------+--------+--------+
|    2017|    2018|    2019|    2020|    2021|
+--------+--------+--------+--------+--------+
| 16481.0| 18094.0| 18500.0| 20157.0| 19828.0|
|153459.0|144421.0|157950.0|175154.0|180214.0|
| 48632.0| 50225.0| 52877.0| 54660.0| 55728.0|
|   175.0|   185.0|   172.0|   144.0|   151.0|
|  4135.0|  4473.0|  4847.0|  5804.0|  6262.0|
|  2522.0|  2498.0|  2920.0|  2857.0|  2754.0|
|   217.0|   193.0|   179.0|   159.0|   149.0|
|   189.0|   197.0|   204.0|   213.0|   212.0|
|289594.0|297832.0|315751.0|328407.0|355502.0|
|   308.0|   303.0|   302.0|   297.0|   302.0|
|    24.0|    19.0|    16.0|    23.0|    22.0|
| 14421.0| 14955.0| 15383.0| 16134.0| 16390.0|
| 29451.0| 30816.0| 32298.0| 32843.0| 33554.0|
|583089.0|602121.0|639659.0|638401.0|634888.0|
|   650.0|   746.0|   797.0|   714.0|   733.0|
|    90.0|   139.0|   121.0|   148.0|   120.0|
|   821.0|   892.0|   975.0|   963.0|   996.0|
|  1875.0|  2149.0|  2212.0|  2342.0|  2240.0|
| 62408.0| 65

In [47]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://plasticpollution.s3.amazonaws.com/MLW_meta_clean.csv"
spark.sparkContext.addFile(url)
mlw_meta_df = spark.read.csv(SparkFiles.get("MLW_meta_clean.csv"), sep=",", header=True, inferSchema=True )

In [48]:
#Show dataframe
mlw_meta_df.show()

+-----------+--------+--------------------+
|generalcode|category|         generalname|
+-----------+--------+--------------------+
|         G1| Plastic|4/6-pack yokes, s...|
|         G3| Plastic|Shopping Bags  in...|
|         G4| Plastic|Small plastic bag...|
|         G5| Plastic|Plastic bags coll...|
|         G7| Plastic|Drink bottles  <=...|
|         G8| Plastic|Drink bottles  >0.5l|
|         G9| Plastic|Cleaner bottles &...|
|        G10| Plastic|Food containers i...|
|        G11| Plastic|Beach use related...|
|        G12| Plastic|Other cosmetics b...|
|        G13| Plastic|Other bottles & c...|
|        G14| Plastic|Engine oil bottle...|
|        G15| Plastic|Engine oil bottle...|
|        G16| Plastic|Jerry cans (squar...|
|        G17| Plastic|Injection gun con...|
|        G18| Plastic|Crates and contai...|
|        G19| Plastic|           Car parts|
|        G21| Plastic|Plastic caps/lids...|
|        G22| Plastic|Plastic caps/lids...|
|        G23| Plastic|Plastic ca

In [49]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://plasticpollution.s3.amazonaws.com/MLW_data_clean.csv"
spark.sparkContext.addFile(url)
mlw_data_df = spark.read.csv(SparkFiles.get("MLW_data_clean.csv"), sep=",", header=True, inferSchema=True,)

In [50]:
#Show dataframe
mlw_data_df.show()

+------------+--------------------+--------------------+-------------+-------------+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+----+----+-----+----+-----+----+----+----+------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+-----+----+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|BeachCountry|           BeachName|    BeachRegionalSea|BeachLength_m|    BeachType|  G1|  G

Connect to the AWS RDS instance and write each DataFrame to its table

In [51]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://plasticpollutiondb.c8tx5tieh7hn.us-east-1.rds.amazonaws.com:5432/plasticpollutiondb"
config = {"user":"plasticpollution",
          "password": password,
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [52]:
# Write Plastic_waste_mismanagement_df to table in RDS
Plastic_waste_mismanagement_df.write.jdbc(url=jdbc_url, table='plastic_waste_mismanagement', mode=mode, properties=config)

In [53]:
# Write pbt_df to table in RDS
pbt_df.write.jdbc(url=jdbc_url, table='pbt', mode=mode, properties=config)

In [54]:
# Write mlw_meta_df to table in RDS
mlw_meta_df.write.jdbc(url=jdbc_url, table='mlw_meta', mode=mode, properties=config)

In [55]:
# Write mlw_data_df to table in RDS
mlw_data_df.write.jdbc(url=jdbc_url, table='mlw_data', mode=mode, properties=config)